In [ ]:
#!pip -q install lightly
#!pip -q install tqdm


In [1]:
import copy
from torch.utils.data import Dataset, DataLoader
import torch
import torchvision
from torch import nn
from PIL import Image
import os

# Set the environment variable
#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments=True'

from lightly.loss import DINOLoss
from lightly.models.modules import DINOProjectionHead
from lightly.models.utils import deactivate_requires_grad, update_momentum
from lightly.transforms.dino_transform import DINOTransform
from lightly.utils.scheduler import cosine_schedule
from lightly.models.modules import SwaVProjectionHead, SwaVPrototypes

class DINO(torch.nn.Module):
    def __init__(self, backbone, input_dim):
        super().__init__()
        self.student_backbone = backbone
        self.student_head = DINOProjectionHead(
            input_dim, 512, 64, 2048, freeze_last_layer=1
        )
        self.teacher_backbone = copy.deepcopy(backbone)
        self.teacher_head = DINOProjectionHead(input_dim, 512, 64, 512)
        deactivate_requires_grad(self.teacher_backbone)
        deactivate_requires_grad(self.teacher_head)

    def forward(self, x):
        y = self.student_backbone(x).flatten(start_dim=1)
        z = self.student_head(y)
        return z

    def forward_teacher(self, x):
        y = self.teacher_backbone(x).flatten(start_dim=1)
        z = self.teacher_head(y)
        return z


f:\Machine learning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
f:\Machine learning\.venv\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [3]:
torch.cuda.empty_cache()


In [4]:
import torch
import torchvision
from torch import nn

from lightly.loss import SwaVLoss
from lightly.models.modules import SwaVProjectionHead, SwaVPrototypes
from lightly.transforms.swav_transform import SwaVTransform

class SwAVModel(torch.nn.Module):
    def __init__(self, backbone, n_prototypes=3000):
        super(SwAVModel, self).__init__()
        self.backbone = backbone
        self.projection_head = SwaVProjectionHead(input_dim=2048, hidden_dim=512, output_dim=128)
        self.prototypes = SwaVPrototypes(input_dim=128, n_prototypes=n_prototypes)

    def forward(self, x):
        features = self.backbone(x).flatten(start_dim=1)
        projections = self.projection_head(features)
        projections = torch.nn.functional.normalize(projections, dim=1)
        assignments = self.prototypes(projections)
        return assignments

    def get_features(self, x):
        features = self.backbone(x).flatten(start_dim=1)
        projections = self.projection_head(features)
        projections = torch.nn.functional.normalize(projections, dim=1)
        return projections


In [5]:
backbone = torch.hub.load('facebookresearch/dino:main', 'dino_vits16', pretrained=False)
input_dim = backbone.embed_dim

Using cache found in /root/.cache/torch/hub/facebookresearch_dino_main


In [6]:
DINO_model = DINO(backbone,input_dim)
Swav_model = SwAVModel(DINO_model)

/opt/conda/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
Swav_model.to(device)

SwAVModel(
  (backbone): DINO(
    (student_backbone): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (blocks): ModuleList(
        (0-11): 12 x Block(
          (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=384, out_features=1152, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=384, out_features=384, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
          (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=384, out_features=1536, bias=True)
            (act): GELU(approximate='none')
            (fc2): Linear(in_features=1536, out_features=384, bias=True)
            (drop): Dr

In [9]:
class CustomImageDataset(Dataset):
    def __init__(self, image_dir, transform=None, n_views=1):
        self.image_dir = image_dir
        self.image_paths = [
            os.path.join(image_dir, f) for f in os.listdir(image_dir)
            if f.lower().endswith(('.jpg', '.png', '.tif'))
        ]
        self.transform = transform
        self.n_views = n_views

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")  # Ensure 3 channels

        if self.transform:
            if self.n_views > 1:
                views = [self.transform(image) for _ in range(self.n_views)]
            else:
                image = self.transform(image)
        else:
            if self.n_views > 1:
                views = [image for _ in range(self.n_views)]
            else:
                pass  # image is already loaded

        dummy_label = 0  # Dummy label (not used)

        if self.n_views > 1:
            return views, dummy_label  # List of views and dummy label
        else:
            return image, dummy_label  # Single image and dummy label


In [10]:
from torchvision import transforms
import os
# Define Image Augmentation (Random Crop, Horizontal Flip, etc.)
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

feature_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Example Dataset (COCO, CIFAR10, ImageNet, or WSI patches)
#train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transforms)
#train_dataset = CustomImageDataset(image_dir='/kaggle/input/tilers/basic_normalized_tiles', transform=train_transforms)# DataLoader
train_dataset = CustomImageDataset(
    image_dir='/kaggle/input/tcga-stad/TCGA_INFORMATIVE_TILES',
    transform=train_transforms,
    n_views=8  # 2 high-res and 6 low-res views
)
feature_dataset = CustomImageDataset(
    image_dir='/kaggle/input/tcga-stad/TCGA_INFORMATIVE_TILES',
    transform=feature_transforms,
    n_views=1
)
feature_loader = DataLoader(
    feature_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4,
)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)

print("DataLoader Ready!")


DataLoader Ready!


In [11]:
for batch_idx, (views, labels) in enumerate(train_loader):
    print(f"Training Batch {batch_idx+1}")
    print(f"Type of views: {type(views)}")  # Should be <class 'list'>
    print(f"Number of views: {len(views)}")  # Should be 8
    print(f"Shape of first view: {views[0].shape}")  # [32, 3, 224, 224]
    break

# Verify subset_feature_loader
for batch_idx, (images, labels) in enumerate(feature_loader):
    print(f"Feature Extraction Batch {batch_idx+1}")
    print(f"Type of images: {type(images)}")  # <class 'torch.Tensor'>
    print(f"Shape of images: {images.shape}")  # [20, 3, 224, 224]
    break

Training Batch 1
Type of views: <class 'list'>
Number of views: 8
Shape of first view: torch.Size([8, 3, 224, 224])
Feature Extraction Batch 1
Type of images: <class 'torch.Tensor'>
Shape of images: torch.Size([32, 3, 224, 224])


In [12]:
criterion = SwaVLoss()
optimizer = torch.optim.Adam(Swav_model.parameters(), lr=0.001)

In [13]:
from tqdm import tqdm

In [ ]:
print("Starting Training")
num_epochs = 10

# Initialize tqdm for epochs
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    Swav_model.train()  # Set model to training mode
    total_loss = 0

    # Initialize tqdm for batches within each epoch
    for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False)):
        # Unpack batch (views, dummy_label)
        views, _ = batch  

        # Diagnostic Prints
        #print(f"\nEpoch {epoch+1}, Batch {batch_idx+1}")
        #print(f"Type of views: {type(views)}")

       

        # Progress Logging
        if batch_idx % 10 == 0:
            print(f'Processed batch {batch_idx+1}/{len(train_loader)}')

        # Normalize prototypes
        Swav_model.prototypes.normalize()

        # Extract features
        multi_crop_features = []
        if isinstance(views, list) or isinstance(views, tuple):
            for view in views:
                feature = Swav_model(view.to(device))
                multi_crop_features.append(feature)
        else:
            multi_crop_features = [Swav_model(views.to(device))]

        #print(f"Extracted {len(multi_crop_features)} multi-crop features.")

        # Split features into high-resolution and low-resolution
        # For n_views=8: first 2 high-res, remaining 6 low-res
        high_resolution = multi_crop_features[:2]  # First 2 views as high-res
        low_resolution = multi_crop_features[2:]   # Remaining 6 views as low-res

        #print(f"High-res features: {[f.shape for f in high_resolution]}")
        #print(f"Low-res features: {[f.shape for f in low_resolution]}")

        # Compute loss
        try:
            loss = criterion(high_resolution, low_resolution)
        except ZeroDivisionError as e:
            print(f"ZeroDivisionError encountered: {e}")
            print("Ensure that n_crops >= 2 and that low_resolution is not empty.")
            continue  # Skips this batch
        except Exception as e:
            print(f"Error computing loss: {e}")
            continue  # Skips this batch

        #print(f"Computed loss: {loss.item()}")
        total_loss += loss.detach()

        # Backward pass and optimization
        loss.backward()

        # Gradient Clipping 
        torch.nn.utils.clip_grad_norm_(Swav_model.parameters(), max_norm=1.0)

        optimizer.step()
        optimizer.zero_grad()
        #print("Performed backward pass and optimizer step.")

       

    # Compute average loss for the epoch
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch: {epoch+1:>02}, Loss: {avg_loss:.5f}")

Starting Training


Epoch 1:   0%|          | 0/2252 [00:00<?, ?it/s]

Processed batch 1/2252



Epoch 1:   0%|          | 10/2252 [00:08<24:06,  1.55it/s]

Processed batch 11/2252


...


Epoch 2:  84%|████████▍ | 1890/2252 [22:21<04:17,  1.41it/s]

Processed batch 1891/2252



Epoch 2:  84%|████████▍ | 1900/2252 [22:28<04:10,  1.41it/s]

Processed batch 1901/2252



Epoch 2:  84%|████████▍ | 1901/2252 [22:29<04:09,  1.41it/s]

In [ ]:
import torch
import numpy as np
from tqdm import tqdm

def extract_features_and_clusters(model, dataloader, device):
    """
    Extract features and SwAV prototype assignments for the dataset.
    
    Args:
        model (SwAV): The trained SwAV model.
        dataloader (DataLoader): DataLoader for the dataset.
        device (torch.device): The device to run the model on.
    
    Returns:
        tuple: Numpy arrays of features (N, feature_dim), cluster assignments (N,), and ground-truth labels (N,)
    """
    features = []
    cluster_assignments = []
    labels = []
    
    model.eval()
    with torch.no_grad():
        for batch_idx, (images, targets) in enumerate(tqdm(dataloader, desc="Extracting Features")):
            images = images.to(device)
    
            # Extract features
            features_batch = model.get_features(images)  # [batch_size, feature_dim]
    
            # Cluster assignments
            logits = model.prototypes(features_batch)  # [batch_size, n_prototypes]
            cluster_ids = torch.argmax(logits, dim=1)  # [batch_size]
    
            # Store results
            features.append(features_batch.cpu().numpy())
            cluster_assignments.append(cluster_ids.cpu().numpy())
            labels.append(targets.cpu().numpy())
    
            if (batch_idx + 1) % 10 == 0:
                print(f'Processed batch {batch_idx+1}/{len(dataloader)}')
    
    features = np.concatenate(features, axis=0)
    cluster_assignments = np.concatenate(cluster_assignments, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, cluster_assignments, labels


In [ ]:
for batch_idx, (images, labels) in enumerate(feature_loader):
    print(f"Feature Extraction Batch {batch_idx+1}")
    print(f"Type of images: {type(images)}")  # <class 'torch.Tensor'>
    print(f"Shape of images: {images.shape}")  # [20, 3, 224, 224]
    break

In [ ]:
features, cluster_assignments, labels = extract_features_and_clusters(Swav_model,feature_loader , device)
np.save('features.npy', features)
np.save('cluster_assignments.npy', cluster_assignments)
np.save('labels.npy', labels)  # If labels are available and meaningful

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

# Optional: For better aesthetics
sns.set(style='darkgrid')

# Initialize t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000, verbose=1)

# Fit and transform the features
features_tsne = tsne.fit_transform(features)
print(f"t-SNE transformed features shape: {features_tsne.shape}")


In [ ]:
# Number of unique clusters
num_clusters = len(np.unique(cluster_assignments))
print(f"Number of unique clusters: {num_clusters}")



# Create a scatter plot
plt.figure(figsize=(12, 10))
palette = sns.color_palette("hsv", num_clusters)

sns.scatterplot(
    x=features_tsne[:,0], y=features_tsne[:,1],
    hue=cluster_assignments,
    palette=palette,
    legend='full',
    alpha=0.7,
    s=50
)

plt.title('SwAV Clustering Visualization with t-SNE')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


In [ ]:
unique_clusters, counts = np.unique(cluster_assignments, return_counts=True)
print("Number of images per cluster:")
for cluster_id, count in zip(unique_clusters, counts):
    print(f"Cluster {cluster_id}: {count} images")

In [ ]:
PARENT_DIR = '/kaggle/working/clusters'
#image_paths=np.load('image_path.npy')
os.makedirs(PARENT_DIR, exist_ok=True)
for idx, (cluster_id, img_path) in enumerate(zip(cluster_assignments, image_paths)):
    try:
        cluster_dir = os.path.join(PARENT_DIR, str(cluster_id))
        os.makedirs(cluster_dir, exist_ok=True)  # Ensure the directory exists
        
        # Define save path
        original_filename = os.path.basename(img_path)
        save_path = os.path.join(cluster_dir, original_filename)
        
        # Open and save the image
        img = Image.open(img_path).convert("RGB")
        img.save(save_path, format='TIFF')  # Change format if needed
        
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")

In [ ]:
import shutil
import os

# Define the path to the clusters directory
clusters_dir = '/kaggle/working/clusters'

# Define the path for the output zip file
output_zip = '/kaggle/working/clusters.zip'

# Check if the clusters directory exists
if os.path.exists(clusters_dir):
    # Create a zip archive of the clusters directory
    shutil.make_archive('/kaggle/working/clusters', 'zip', clusters_dir)
    print("Clusters directory has been compressed successfully.")
else:
    print(f"The directory {clusters_dir} does not exist.")

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
import math

# Load cluster assignments and features
cluster_assignments = np.load('cluster_assignments.npy')
image_paths = feature_dataset.image_paths  # Paths from the dataset

# Function to visualize tiles for a specific cluster
def visualize_cluster(cluster_id, num_images=9):
    indices = np.where(cluster_assignments == cluster_id)[0]
    selected_indices = indices[:num_images]  # Select up to num_images from the cluster

    # Dynamically calculate rows and columns
    cols = 5  # You can adjust this to your preference
    rows = math.ceil(len(selected_indices) / cols)

    plt.figure(figsize=(cols * 3, rows * 3))
    plt.suptitle(f"Cluster {cluster_id}", fontsize=16)

    for i, idx in enumerate(selected_indices):
        img_path = image_paths[idx]
        img = Image.open(img_path).convert("RGB")

        plt.subplot(rows, cols, i + 1)  # Adjust grid dynamically
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Image {idx}")
    
    plt.tight_layout()
    plt.show()

# Visualize cluster 0
visualize_cluster(cluster_id=2973, num_images=50)

# Visualize more clusters as needed
# visualize_cluster(cluster_id=1, num_images=9)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Load extracted features and labels
features = np.load('features.npy')             # Shape: [N, feature_dim]
labels = np.load('labels.npy')                 # Shape: [N]
cluster_assignments = np.load('cluster_assignments.npy')  # Shape: [N]

# Define the unique classes
classes = np.unique(cluster_assignments)

# Train-Test Split using original 1D labels
X_train, X_test, y_train, y_test = train_test_split(
    features,
    cluster_assignments,        # Use 1D labels here
    test_size=0.3,
    random_state=42
)

# Train a Logistic Regression Classifier
clf = LogisticRegression(
    max_iter=1000,
    multi_class='ovr',          # One-vs-Rest strategy for multi-class
    solver='lbfgs',
    class_weight='balanced'
)
clf.fit(X_train, y_train)

# Predict probabilities
y_prob = clf.predict_proba(X_test)  # Shape: [N_test, num_classes]

# Binarize y_test for ROC Curve (after prediction)
y_test_binarized = label_binarize(y_test, classes=classes)  # Shape: [N_test, num_classes]
unique_train_classes = np.unique(y_train)
missing_classes = set(classes) - set(unique_train_classes)

if missing_classes:
    print(f"The following classes are missing in the training set: {missing_classes}")
    # Handle missing classes as needed
else:
    print("All classes are represented in the training set.")
# Plot the ROC Curve
plt.figure(figsize=(10, 8))

for i, class_label in enumerate(classes):
    # Compute ROC curve and AUC for each class
    fpr, tpr, _ = roc_curve(y_test_binarized[:, i], y_prob[:, i])
    roc_auc = auc(fpr, tpr)
    
    plt.plot(fpr, tpr, label=f'Class {class_label} (AUC = {roc_auc:.2f})')

# Plot settings
plt.plot([0, 1], [0, 1], color='gray', linestyle='--', label='Random Guessing')  # Diagonal line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(alpha=0.5)
plt.show()
